## Overview

To facilitate graph analysis, we need to support sharing layouts between graphs, something I haven't seen done very well elsewhere.  In particular, it should be possible to:

* &#x2713; "Pin" a few key vertices while letting the layout handle the remaining vertices.
* &#x2713; Render a graph using another graph's vertices, whether it is a superset or subset of the original.
* &#x2713; An implicit requirement is the ability to perform a layout on some vertices while leaving others unchanged.
* &#x2713; Another implicit requirement is the ability to supply disconnected vertex ids to be merged with the induced ids. 

So the process to layout graph $G_1$, given an (optional) layout of graph $G_0$ is (I think):

* Extract the edges for $G_1$.
    * Either an $E \times 2$ matrix, or
    * Two $|E|$ vectors.
* Identify the vertex ids $V_1$ induced by those edges.
* Merge optional user-supplied vertex ids with $V_1$ (to allow for disconnected vertices).
* Create a set of completely masked vertex coordinates, $V_{C_1}$.
* If the user supplied an "other" toyplot.layout.GraphLayout object from $G_0$, use the $V_0$ vertex ids to merge $V_{C_0}$ coordinates into $V_{C_1}$.
* If the user supplied a set of external vertex coordinates $V_{C_{external}}$ with length $|V_1|$, merge them into $V_{C_1}$.
* Apply a layout algorithm to any remaining masked values in $V_{C_1}$.

Allowed input formats:

* toyplot.graph(edges, [vids], [olayout], [vcoordinates])
* toyplot.graph(sources, targets, [vids], [olayout], [vcoordinates])

In [1]:
import numpy
import toyplot.color
import toyplot.generate

In [2]:
numpy.random.seed(1234)

In [3]:
colormap = toyplot.color.LinearMap(toyplot.color.Palette(["white", "yellow", "red"]))

In [4]:
G0 = toyplot.generate.prufer_tree(numpy.random.choice(4, 12))
G1 = G0[:-4]
G2 = G0[4:]

In [5]:
canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.axes(grid=(1, 2, 0), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G0,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black")

vcoordinates = numpy.ma.masked_all((mark.vcount, 2))
vcoordinates[0] = (-1, 0)
vcoordinates[1] = (0, 0)
vcoordinates[2] = (1, 0)

axes = canvas.axes(grid=(1, 2, 1), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G0,
    vcoordinates=vcoordinates,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black");

print "Pinned Vertices:"

INFO:toyplot:Graph layout time: 61.9859695435 ms
INFO:toyplot:Graph layout time: 44.4211959839 ms


Pinned Vertices:


In [6]:
#G0_layout = toyplot.layout.graph(G0)

canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.axes(grid=(1, 2, 0), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G0,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black")

axes = canvas.axes(grid=(1, 2, 1), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G1,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black");

print "Shared Layout (Subset):"

INFO:toyplot:Graph layout time: 46.9298362732 ms
INFO:toyplot:Graph layout time: 43.4799194336 ms


Shared Layout (Subset):


In [7]:
canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.axes(grid=(1, 2, 0), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G2,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black")

axes = canvas.axes(grid=(1, 2, 1), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G0,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black");

print "Shared Layout (Superset):"

INFO:toyplot:Graph layout time: 46.7410087585 ms
INFO:toyplot:Graph layout time: 44.3050861359 ms


Shared Layout (Superset):


In [8]:
extra_vids = numpy.arange(50, 55)

canvas = toyplot.Canvas(width=1000, height=500)

axes = canvas.axes(grid=(1, 2, 0), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G2,
    extra_vids,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black")

axes = canvas.axes(grid=(1, 2, 1), show=False)
axes.aspect = "expand-domain"
mark = axes.graph(
    G0,
    extra_vids,
    olayout = mark,
    vcolor=colormap,
    vstyle={"stroke":"black"},
    vsize=20,
    ecolor="black",
    eopacity=0.2,
);
axes.text(mark.vcoordinates.T[0], mark.vcoordinates.T[1], mark.vids, color="black");

print "Shared Layout (Disconnected Vertices):"

INFO:toyplot:Graph layout time: 46.43201828 ms
INFO:toyplot:Graph layout time: 45.9299087524 ms


Shared Layout (Disconnected Vertices):
